# Curso: Bioestatística — Delineamento hierárquico
## Autores: Sandro da Silva Camargo e Fernando Cardoso

**Problema**:
* Foi projetado um estudo de comportamento animal para verificar os potenciais efeitos depressores (Y) de 2 produtos farmacêuticos (Tratamentos);
* 30 ratos foram utilizados no grupo de estudo, com 10 ratos atribuı́dos ao acaso a cada produto (T1 e T2) e 10 para o grupo de controle (C), que recebeu placebo.
* A resposta foi medida 2 vezes (n=2) em cada animal.

A base de dados está disponível [aqui](https://github.com/Sandrocamargo/biostatistics/blob/master/datasets/hierarquico-ratos.txt).

Abra este código no seu google colab [clicando aqui](https://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_08_Delineamento_Hierarquico.ipynb).

# Delineamento completamente casualizado com subamostragem


# Carga de bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

# Carga e inspeção de dados

In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/hierarquico-ratos.txt")

# Conversão para categorias
dados["tratamento"] = dados["tratamento"].astype("category")
dados["rato"] = dados["rato"].astype("category")
dados["rato_trt"] = dados["rato.trt"].astype("category")

dados.info()
dados.describe()

# Ajuste do modelo ANOVA

In [ ]:
# --- 3. ANOVA simples ---
modelo_aov = smf.ols("y ~ C(tratamento)", data=dados).fit()
print(modelo_aov.summary())
anova_table = sm.stats.anova_lm(modelo_aov, typ=2)
print("\n=== ANOVA ===")
print(anova_table)

** 📊 Interpretação da regressão**
|Parâmetro	|Coef.	|Interpretação|
|:--|:--|:--|
|Intercept |= 47.20	|Representa a média do grupo controle.	|
|C(tratamento)[T.T1]| = -11.55	|A média do grupo T1 é 11,55 unidades menor que a do controle, diferença altamente significativa (p < 0.001).	|
|C(tratamento)[T.T2]| = -10.00	|A média do grupo T2 é 10 unidades menor que a do controle, também altamente significativa (p < 0.001).	|

➡️ Assim, as médias estimadas são aproximadamente:

* Controle: 47.20
* T1: 35.65
* T2: 37.20

**🧠 Significância e ajuste do modelo**
|Indicador	|Valor	|Interpretação|
|:--|:--|:--|
|R² |= 0.453	|O modelo explica cerca de 45,3% da variação total em y.	|
|F(2,57) |= 23.59, p < 0.0001	|O fator tratamento é globalmente significativo — ou seja, pelo menos um dos tratamentos difere do outro.	|
|Durbin-Watson |= 2.64	|Sem autocorrelação relevante entre resíduos (valor próximo de 2 é ideal).	|
|Omnibus e Jarque-Bera |(p > 0.3)	|Indicam normalidade dos resíduos, o que valida o modelo ANOVA.	|

**🔬 ANOVA**
|Fonte de variação	|SQ	|gl	|F	|p|
|:--|:--|:--|:--|:--|
|Tratamento	|1572,03	|2	|23.59|	3.4e−08|
|Resíduo	|1898,95	|57	|—|	—|

➡️ O valor de p < 0.001 confirma que há diferenças significativas entre os grupos.

A proporção $\frac{SQ_{Tratamentos}}{SQ_{Total}} = \frac{1572}{1572+1899} = 45\%$, coincide com o R², reforçando a coerência do modelo.

**📈 Conclusão**

* O tratamento tem efeito estatisticamente significativo sobre a variável resposta.
* T1 e T2 apresentam valores menores que o controle, e ambos diferem de forma significativa.
* T1 e T2 parecem ter efeitos semelhantes entre si, dado que suas estimativas são próximas (−11,55 vs −10,00).

➡️ Seria interessante confirmar com um teste de comparações múltiplas (Tukey) se T1 e T2 diferem entre si.

# Gráficos exploratórios

In [ ]:
sns.boxplot(x="tratamento", y="y", data=dados)
plt.title("Boxplot dos tratamentos")
plt.show()

In [ ]:
plt.scatter(dados["y"], modelo_aov.fittedvalues,
            c=dados["tratamento"].cat.codes, cmap="viridis", s=70)
plt.xlabel("Valores observados")
plt.ylabel("Valores preditos")
corr = np.corrcoef(dados["y"], modelo_aov.fittedvalues)[0, 1]
plt.title(f"ANOVA ajustada\nCorrelação = {corr:.4f}")
plt.plot([dados["y"].min(), dados["y"].max()],
         [dados["y"].min(), dados["y"].max()], "r--")
plt.show()

# Teste de Tukey

In [ ]:
tukey = pairwise_tukeyhsd(endog=dados["y"], groups=dados["tratamento"], alpha=0.05)
print("\n=== Teste de Tukey ===")
print(tukey)

**🧩 Lógica do teste**

O Tukey HSD (_Honest Significant Difference_) compara todas as médias par a par, controlando o erro tipo I global (FWER = 0.05).

Cada linha da tabela mostra:
* group1 / group2 → pares comparados
* meandiff → diferença entre as médias (group2 − group1)
* p-adj → valor de p ajustado
* lower / upper → limites do intervalo de confiança da diferença
* reject → se a hipótese nula (igualdade entre médias) foi rejeitada (True = diferença significativa)

**📊 Interpretação dos resultados**
|Comparação	|Diferença média	|p-ajustado	|IC95%	|Diferença significativa?	|Interpretação|
|:--|:--|:--|:--|:--|:--|
|C vs T1	|−11.55	|0.000	|[−15.94, −7.16]|	✅ Sim	|O grupo T1 tem média significativamente menor que o controle.|
|C vs T2|	−10.00	|0.000	|[−14.39, −5.61]|	✅ Sim	|O grupo T2 também tem média significativamente menor que o controle.|
|T1 vs T2|	+1.55	|0.674	|[−2.84, +5.94]|	❌ Não	|T1 e T2 não diferem significativamente entre si.|

**🧠 Conclusão geral**
* Há diferença estatisticamente significativa entre o grupo controle e ambos os tratamentos (T1 e T2).
* Os tratamentos T1 e T2 produzem respostas menores que o controle, com redução média de cerca de 10 a 12 unidades.
* Não há diferença significativa entre T1 e T2, o que sugere que os dois têm efeitos equivalentes sobre a variável resposta.

**📈 Resumo interpretativo**

$C (controle) > T1 ≈ T2$

* Em termos práticos, isso indica que os dois tratamentos reduzem a resposta de forma semelhante e ambos diferem claramente do controle, mas não entre si.

# Modelo Misto (efeito aleatório: rato_trt)

In [ ]:
md = smf.mixedlm("y ~ C(tratamento)", data=dados, groups=dados["rato_trt"])
mfit = md.fit(reml=True)
print("\n=== Modelo Misto (LME) ===")
print(mfit.summary())

**📊 Resultados fixos (efeitos médios dos tratamentos)**

| Parâmetro | Estimativa | Erro Padrão | z | p-valor | IC95% | Interpretação |
|------------|-------------|--------------|----|---------|----------------|--|
| Intercept | 47.20 | 1.82 | 25.94 | $<$0.001 | [43.63, 50.77] | Média estimada do grupo Controle (C). |
| T1 | −11.55 | 2.57 | −4.49 | $<$0.001 | [−16.59, −6.51] | O tratamento T1 reduz a média em cerca de 11,6 unidades em relação ao controle. Diferença altamente significativa. |
| T2 | −10.00 | 2.57 | −3.89 | $<$0.001 | [−15.04, −4.96] | O tratamento T2 reduz a média em cerca de 10 unidades em relação ao controle. Diferença altamente significativa. |

**✅ Conclusão**
* Ambos os tratamentos (T1 e T2) apresentam efeito negativo significativo comparado ao controle.
* As magnitudes das reduções são próximas — reforçando que T1 ≈ T2 < Controle.

**🧩 Resultados aleatórios**
|Componente	|Variância	|Interpretação|
|:--|:--|:--|
|Group |Var = 31.241		|Há variabilidade considerável entre grupos (por exemplo, entre ratos dentro de cada tratamento). Isso justifica o uso do modelo misto. O modelo simples (ANOVA tradicional) ignoraria essa dependência.|

* O fato de a variância de grupo ser diferente de zero mostra que as subamostras dentro do mesmo grupo são correlacionadas, e o modelo misto captura essa correlação adequadamente.

**🧠 Interpretação geral**
* O modelo convergiu bem (indicando ajuste estável).
* Há diferenças significativas entre o controle e ambos os tratamentos.
* A variabilidade entre grupos é importante e foi devidamente considerada (Group Var ≈ 31).
* Não há indicação de problemas de ajuste (convergência ou erro de estimação).

**📈 Resumo final**

$Controle (C)>T1 ≈ T2$

**Conclusão prática:**
* Mesmo após considerar a variação entre grupos (ratos, plantas etc.), ambos os tratamentos reduziram significativamente a resposta média em relação ao controle, e não há diferença relevante entre T1 e T2.

# Diagnóstico e ajuste do modelo misto

In [ ]:
plt.scatter(dados["y"], mfit.fittedvalues,
            c=dados["tratamento"].cat.codes, cmap="plasma", s=70)
plt.xlabel("Valores observados")
plt.ylabel("Valores preditos (LME)")
corr2 = np.corrcoef(dados["y"], mfit.fittedvalues)[0, 1]
plt.title(f"Modelo misto ajustado\nCorrelação = {corr2:.4f}")
plt.plot([dados["y"].min(), dados["y"].max()],
         [dados["y"].min(), dados["y"].max()], "r--")
plt.show()

# Histograma dos resíduos
plt.hist(mfit.resid, bins=10, edgecolor='black')
plt.title("Distribuição dos resíduos (modelo misto)")
plt.show()

# Médias ajustadas (LS-means aproximadas)
Predição ao valor médio dos efeitos aleatórios

In [ ]:
means = (dados.groupby("tratamento", observed=False)["y"]
         .agg(["mean", "std", "count"])
         .rename(columns={"mean": "lsmean"}))
print("\n=== Médias ajustadas (aproximadas) ===")
print(means)

**📊 O que são LS-means**

As LS-means (Least Squares Means) representam as médias estimadas para cada tratamento, corrigidas pelos efeitos do modelo (por exemplo, considerando o efeito aleatório de grupos e eventuais covariáveis).
Elas são preferíveis às médias simples quando há estrutura hierárquica ou desequilíbrio nos dados.

**📈 Interpretação dos valores**
|Tratamento	|Média ajustada (LS-mean)|	Desvio-padrão	|n|
|:--|:--|:--|:--|
|C (Controle)	|47.20	|5.01	|20|
|T1	|35.65	|6.38	|20|
|T2	|37.20	|5.85	|20|

Significado prático:
* O grupo Controle (C) apresenta a maior resposta média ajustada (≈47,2).
* O grupo T1 apresenta a menor média (≈35,6).
* O grupo T2 é intermediário, com média ≈37,2, próxima de T1.

**🧠 Comparações qualitativas**

Com base nesses valores:
* A diferença C − T1 ≈ 11,6 e C − T2 ≈ 10,0 confirma os resultados anteriores da ANOVA e do modelo misto. Ambos os tratamentos reduzem significativamente a resposta média em relação ao controle.
* A diferença T1 − T2 ≈ −1,55 é pequena, o que explica por que não foi significativa no teste de Tukey.

**💡 Conclusão geral**
* O Controle (C) mantém a resposta mais alta e significativamente diferente dos demais.
* Os tratamentos T1 e T2 têm efeitos semelhantes entre si, ambos reduzindo a resposta.
* As médias ajustadas reforçam a robustez do resultado obtido no modelo misto, mesmo controlando variação entre grupos, o padrão C > T1 ≈ T2 se mantém.

# Contrastes manuais

In [ ]:
trat1_mean = means.loc["T1", "lsmean"]
trat2_mean = means.loc["T2", "lsmean"]
ctrl_mean = means.loc["C", "lsmean"]
diff = trat1_mean - ctrl_mean
print(f"\nContraste T1 vs C = {diff:.3f}")
diff = trat2_mean - ctrl_mean
print(f"\nContraste T2 vs C = {diff:.3f}")
diff = trat1_mean - trat2_mean
print(f"\nContraste T1 vs T2 = {diff:.3f}")

**📊 Resultados dos contrastes**

|Comparação	|Diferença (Contraste)|	Interpretação|
|:--|:--|:--|
|T1 – C |= -11.55	|O grupo T1 tem média 11,55 unidades menor que o grupo Controle (C).	|
|T2 – C |= -10.00	|O grupo T2 tem média 10,00 unidades menor que o grupo Controle (C).	|
|T1 – T2 |= -1.55	|O grupo T1 tem média 1,55 unidade menor que o grupo T2 — uma diferença pequena e provavelmente não significativa.	|

**🧠 Interpretação conceitual**

* Tanto T1 quanto T2 apresentaram reduções significativas em relação ao controle (efeito negativo substancial).
* As magnitudes das reduções são semelhantes (−11,55 e −10,00), sugerindo que ambos os tratamentos são eficazes em produzir o efeito observado (por exemplo, redução de uma variável fisiológica, de um marcador, etc.).
* A diferença entre T1 e T2 é pequena e próxima de zero, indicando que os dois tratamentos têm efeitos comparáveis.

**✅ Conclusão prática**

* Há diferença significativa entre o controle e os tratamentos, confirmando o efeito do fator “tratamento”.
* Não há diferença relevante entre T1 e T2, o que sugere que qualquer um dos dois tratamentos produz resultado semelhante.

# Exemplo de ANOVA com fatores adicionais

In [ ]:
modelo2 = smf.ols("y ~ C(rato) + C(tratamento) + C(rato_trt)", data=dados).fit()
anova2 = sm.stats.anova_lm(modelo2, typ=2)
print("\n=== ANOVA com fatores adicionais ===")
print(anova2)

**📊 Interpretação dos efeitos**
|Fonte de variação	|SQ |gl	|F	|p-valor	|Interpretação|
|:--|:--|:--|:--|:--|:--|
|C(rato)	|514.09	|9	|15.37	|5.6×10⁻⁹	|Diferenças significativas entre ratos, indicando que o desempenho dos animais varia consideravelmente.|
|C(tratamento)	|427.92	|2	|57.57	|5.4×10⁻¹¹	|Diferenças altamente significativas entre tratamentos, ou seja, os medicamentos influenciam a resposta.|
|C(rato_trt)	|18631.20	|29	|172.86	|2.1×10⁻²⁶	|Grande variação entre subamostras dentro de tratamento, possivelmente refletindo heterogeneidade interna (variação entre medições dentro de cada rato).|
|Residual	|111.50	|30	|—	|—	|Variação residual mínima após considerar os efeitos acima.|

**🧠 Interpretação geral**

* O fator tratamento apresenta efeito altamente significativo, confirmando que há diferenças reais entre os grupos experimentais.
* O fator rato também é significativo, indicando efeito aleatório importante (animais diferem em sua resposta basal).
* A variação dentro de cada rato_trt é muito grande, o que justifica a necessidade de um modelo hierárquico (ou misto) para modelar essa estrutura corretamente.

**💬 Conclusão prática**

O delineamento e a análise mostram que:
* Há diferenças significativas entre os tratamentos aplicados.
* Há também heterogeneidade entre os indivíduos (ratos), o que precisa ser controlado.
* A subamostragem dentro de tratamento (rato_trt) contribui com grande parte da variabilidade total, reforçando que o modelo simples (ANOVA tradicional) subestima o erro experimental — e que o modelo misto (LME) é a abordagem estatística correta.